# **Extract**

In [0]:
#df= spark.read.parquet("/Volumes/workspace/dbs/dbs_volume/house-price.parquet")
df.show(10)

# **Load**

In [0]:
# Check missing values count per column
from pyspark.sql.functions import col, count, when

df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

# Example: Fill missing LotFrontage with median or drop rows with critical nulls
#median_lotfrontage = df.approxQuantile("LotFrontage", [0.5], 0.01)[0]

#df_cleaned = df.fillna({"LotFrontage": median_lotfrontage})

### New column

In [0]:
df = df.withColumn("total_rooms", col("bedrooms") + col("bathrooms"))

In [0]:
from pyspark.sql.functions import when

# Convert yes/no columns to 1/0
binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

for c in binary_cols:
    df = df.withColumn(c, when(col(c) == 'yes', 1).otherwise(0))

In [0]:
# Example: simple label encoding
df = df.withColumn("furnishingstatus_index",
    when(col("furnishingstatus") == "furnished", 2)
    .when(col("furnishingstatus") == "semi-furnished", 1)
    .otherwise(0)
)

# Load

In [0]:
df.write.mode("overwrite").saveAsTable("dbs.cleaned_house_price")

In [0]:
%sql
select * from dbs.cleaned_house_price